In [1]:
# import relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import plotly.express as px
import json
from shapely.geometry import Point, Polygon
import geopandas as gpd
import re
import geobr
import matplotlib.pyplot as plt
import warnings
from unidecode import unidecode
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")

/Users/fihashim/miniconda3/envs/py310/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Read CSVs of dataset
custom = pd.read_csv('olist_customers_dataset.csv', sep=',')
orders = pd.read_csv('olist_orders_dataset.csv', sep=',')
order_items = pd.read_csv('olist_order_items_dataset.csv', sep=',')
order_reviews = pd.read_csv('olist_order_reviews_dataset.csv', sep=',')
geolocation = pd.read_csv('olist_geolocation_dataset.csv', sep=',')
order_payments = pd.read_csv('olist_order_payments_dataset.csv', sep=',')
products_df = pd.read_csv('olist_products_dataset.csv', sep=',')
sellers_df = pd.read_csv('olist_sellers_dataset.csv', sep=',')
translations_df = pd.read_csv('product_category_name_translation.csv', sep=',')

##### 7. How many sellers are active on Olist, and how does this number change over time?
Sellers are considered active if they have made a sale within a 30-day period


In [3]:
sellers_order_items_orders = sellers_df.merge(order_items, how='left', on='seller_id').merge(orders, how='left', on='order_id')
sellers_order_items_orders = sellers_order_items_orders[sellers_order_items_orders['order_status'] == 'delivered']
sellers_order_items_orders = sellers_order_items_orders.drop_duplicates(subset=['seller_id'], keep='first')

# sellers are considered active if they make a make within the last 30 days which have been delivered.
sellers_order_items_orders['before_purchase'] = pd.to_datetime(sellers_order_items_orders['order_purchase_timestamp'].min()) - pd.Timedelta(30, unit='d')
sellers_order_items_orders['after_purchase'] = pd.to_datetime(sellers_order_items_orders['order_purchase_timestamp'].max())

sellers_order_items_orders.loc[(sellers_order_items_orders['order_purchase_timestamp'] >= sellers_order_items_orders['before_purchase']) & (sellers_order_items_orders['order_purchase_timestamp'] <= sellers_order_items_orders['after_purchase']), 'seller_status'] = 'active'
sellers_order_items_orders.loc[(sellers_order_items_orders['order_purchase_timestamp'] <= sellers_order_items_orders['before_purchase']) & (sellers_order_items_orders['order_purchase_timestamp'] >= sellers_order_items_orders['after_purchase']), 'seller_status'] = 'not_active'
